In [1]:
from discog_func import *

token = 'eGkJSFluvBIeQCEmsueVEMZIabFbDLaPFIDcKFkg'
artist = 'Myron Walden'

collabs = discog(artist,token)
top_five = collabs[:5]

In [20]:
import plotly.plotly as py

node_trace, edge_trace, annots = construct_g(artist,top_five,token)
fig = Figure(data=Data([edge_trace, node_trace]),
             layout=Layout(
                title=artist + "'s Top Five Collaborators", #'<br>Network graph made with Python',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations= annots,
                xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False)))

py.iplot(fig, filename=artist+"'s Collaborators")